In [ ]:
# Hypothesis Test Regarding Score and Thumb-up
import pandas as pd
from scipy.stats import f_oneway

df = pd.read_csv("Hypothesis_Test_Score_And_Thumb-up.csv")

group_1 = df[df['THUMBS_GROUP'] == '1-5']['SCORE']
group_2 = df[df['THUMBS_GROUP'] == '6-20']['SCORE']
group_3 = df[df['THUMBS_GROUP'] == '20+']['SCORE']

# One-Way ANOVA
f_stat, p_val = f_oneway(group_1, group_2, group_3)
print("F-statistic:", f_stat)
print("p-value:", p_val)


F-statistic: 1.756169789624281
p-value: 0.17297117872082665


In [15]:
import pandas as pd

df = pd.read_csv("chatgpt_reviews_partial_100000.csv")

df = df[df['content'].notna()]
df['content'] = df['content'].astype(str).str.strip()
df = df[df['content'].str.len() > 0]
def has_non_ascii(text):
    return any(ord(char) > 127 for char in text)

df['has_non_ascii'] = df['content'].apply(has_non_ascii)

non_ascii_ratio = df['has_non_ascii'].mean()
print(f"Percentage of reviews with non-ASCII characters: {non_ascii_ratio:.2%}")

df['content_length'] = df['content'].str.len()
emoji_like = df[(df['has_non_ascii']) & (df['content_length'] < 5)]
print(f"Potential emoji-only or non-informative reviews: {len(emoji_like)}")


Percentage of reviews with non-ASCII characters: 21.68%
Potential emoji-only or non-informative reviews: 2900


## Product Team Centric Analysis

In [2]:
pip install "snowflake-connector-python[pandas]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 1.8 MB/s eta 0:00:00
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached pandas-2.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached s3transfer-0.13.1-py3-none-any.whl.metadata (1.7 kB)
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 26.2 MB/s eta 0:00:00a 0:00:01
Using cached pandas-2.3.1-cp312-cp312-macosx_11_0_arm64.whl (10.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
Using cached s3transfer-0.13.1-py3-none-any.whl (85 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
  

In [3]:
import snowflake.connector
print(snowflake.connector.__version__)


3.17.1


In [10]:
!pip install -U scikit-learn snowflake-connector-python pyarrow


  Using cached pyarrow-21.0.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 16.6 MB/s eta 0:00:00a 0:00:01
Using cached pyarrow-21.0.0-cp312-cp312-macosx_12_0_arm64.whl (31.2 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
# analyze_topics

import os, re, numpy as np, pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from snowflake.connector import connect
from sklearn.feature_extraction import text as sk_text

# Settings & Parameters
DB_NAME       = "CHATGPT_REVIEWS_DB"
SCHEMA_NAME   = "STAGING"
VIEW_NAME     = "V_LOW_SCORE_CLEAN"

TABLE_TOPICS   = "TOPICS_KEYWORDS"
TABLE_EXAMPLES = "TOPICS_EXAMPLES"
TABLE_SUMMARY  = "TOPICS_SUMMARY"

K_RANGE          = range(6, 13)      # Choose from 6 to 12
TOPN_WORDS       = 12                
EXAMPLES_PER_TP  = 5
MIN_DF           = 12            # A word needs to be presented in at least 12 reviews to be kept 
MAX_DF           = 0.40
MAX_FEATURES     = 50000
EN_PROP_MIN      = 0.60

MMR_LAMBDA       = 0.7           # The larger the value, the more emphasis on relevance; the smaller the value, the more emphasis on diversity.
DEDUP_JACCARD    = 0.90          # Delete duplicate (when similarity is greater than or equal to 0.9, we see it as duplicated reviews) 


# Adjust when needed
DOMAIN_STOP = {
    "app","apps","chatgpt","openai","ai","gpt","version","versions","update","updated",
    "fix","fixed","issue","issues","problem","problems","bug","bugs","please","pls",
    "thanks","thank","hi","hello","team","dear","experience","experiences",
    "nice","good","bad","very","best","worst","useful","useless","helpful","not",
    "really","actually","also","ever","always","never","still","just","well","ok",
    "work","works","working","worked","proper","properly","application"
}
STOP_WORDS = list(sk_text.ENGLISH_STOP_WORDS.union(DOMAIN_STOP))

CANON_MAP = {
    r"\blog in\b": "login", r"\blogin\b": "login", r"\bsign in\b": "login",
    r"\b2fa\b": "mfa", r"\b2-factor\b": "mfa", r"\b2 factor\b": "mfa",
    r"\bslow\b": "lag", r"\blaggy\b": "lag", r"\blag\b": "lag",
    r"\bprice\b": "pricing", r"\bcharged?\b": "billing",
    r"\brefunds?\b": "refund", r"\bsubscription\b": "subscribe",
    r"\bcrash(es|ed|ing)?\b": "crash",
    r"\bdoesn['’]?t work\b": "not_working",
    r"\bisn['’]?t working\b": "not_working",
    r"\bnot working\b": "not_working",
}

NAME_RULES = [
    ("Login / Account",     ["login","account","password","otp","mfa","verification"]),
    ("Billing / Subscription",["billing","subscribe","payment","pricing","refund","charge"]),
    ("Performance / Crash", ["lag","slow","freeze","loading","crash","latency"]),
    ("Answer Quality",      ["answer","response","quality","accuracy","wrong","hallucination"]),
    ("Access / Region",     ["region","country","available","access","blocked","unsupported"]),
    ("UI / Usability",      ["ui","ux","button","menu","dark mode","layout"]),
]

# Snowflake 
def load_env():
    load_dotenv(override=True)
    for k in ["SNOWFLAKE_USER","SNOWFLAKE_PASSWORD","SNOWFLAKE_ACCOUNT"]:
        if not os.getenv(k): raise RuntimeError(f"Missing .env key: {k}")
    return {
        "user": os.getenv("SNOWFLAKE_USER"),
        "password": os.getenv("SNOWFLAKE_PASSWORD"),
        "account": os.getenv("SNOWFLAKE_ACCOUNT"),
        "role": os.getenv("SNOWFLAKE_ROLE", "SYSADMIN"),
        "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE", "COMPUTE_WH"),
        "database": os.getenv("SNOWFLAKE_DATABASE", DB_NAME),
        "schema": os.getenv("SNOWFLAKE_SCHEMA", SCHEMA_NAME),
    }

def connect_sf(cfg):
    conn = connect(user=cfg["user"], password=cfg["password"], account=cfg["account"],
                   warehouse=cfg["warehouse"], database=cfg["database"],
                   schema=cfg["schema"], role=cfg["role"])
    with conn.cursor() as cs:
        cs.execute(f"USE ROLE {cfg['role']}")
        cs.execute(f"USE WAREHOUSE {cfg['warehouse']}")
        cs.execute(f"USE DATABASE {DB_NAME}")
        cs.execute(f"USE SCHEMA {SCHEMA_NAME}")
    return conn

def fetch_df(conn):
    sql = f"""
      SELECT review_id, content, thumbs_up_count, app_version, review_time
      FROM {DB_NAME}.{SCHEMA_NAME}.{VIEW_NAME}
    """
    df = pd.read_sql(sql, conn)
    df.columns = [c.lower() for c in df.columns]
    return df

# Data Cleaning 
# Data cleaning is before TF-IDF Vectorization
def english_prop(s: str) -> float:
    s = str(s or "")
    letters = sum(ch.isalpha() for ch in s)
    en_letters = sum('a' <= ch.lower() <= 'z' for ch in s)
    return (en_letters / letters) if letters else 0.0

def canon_replace(text: str) -> str:
    t = text
    for pat, rep in CANON_MAP.items(): t = re.sub(pat, rep, t)
    return t

def clean_text(s: str) -> str:
    t = str(s or "").lower()
    t = re.sub(r"http\S+|www\S+", " ", t)         # Get rid of URL
    t = canon_replace(t)                          # Synonym normalization
    t = re.sub(r"[^a-z0-9\s']", " ", t)          # English letters / digits / spaces / apostrophes only
    t = re.sub(r"\s+", " ", t).strip()
    return t

# Similarity & MMR (select representative reviews)
def _tokens_for_jaccard(text: str):
    return {w for w in re.findall(r"[a-zA-Z]{2,}", (text or "").lower()) if w not in STOP_WORDS}

def jaccard(a: set, b: set) -> float:
    if not a or not b: return 0.0
    return len(a & b) / len(a | b)

def mmr_select(texts, rel_scores, top_k, lam=MMR_LAMBDA):
    chosen, cand = [], list(range(len(texts)))
    token_sets = [_tokens_for_jaccard(t) for t in texts]
    if not cand: return []
    first = int(np.argmax(rel_scores)); chosen.append(first); cand.remove(first)
    while len(chosen) < min(top_k, len(texts)) and cand:
        best_i, best_score = None, -1e9
        for i in cand:
            sim_to_chosen = 0.0 if not chosen else max(jaccard(token_sets[i], token_sets[j]) for j in chosen)
            score = lam * rel_scores[i] - (1 - lam) * sim_to_chosen
            if score > best_score: best_score, best_i = score, i
        chosen.append(best_i); cand.remove(best_i)
    return chosen

# Topics Naming 
def _is_english_phrase(s: str) -> bool:
    # Allow only A–Z and spaces (compatible with bigram/trigram)
    return bool(re.fullmatch(r"[A-Za-z]+(?: [A-Za-z]+)*", s or ""))

def auto_name(keywords: list[str]) -> str:
    kw_join = " ".join(keywords)
    for name, needles in NAME_RULES:
        if any(n in kw_join for n in needles): return name
    # Fallback: use English phrases only
    eng = [w for w in keywords if _is_english_phrase(w)]
    return ", ".join(eng[:3]).title() if eng else "General"

# Choose K(6 to 12) 
def score_topics(H):
    sim = cosine_similarity(H)            # we want this to be small (similarity between different topics should be small)
    np.fill_diagonal(sim, 0.0)
    inter_sim = sim.mean()
    sparsity = (H < (H.mean(axis=1, keepdims=True))).mean()  # we want this to be large (the fewer words that can represent a topic, the better)
    return (1 - inter_sim) * 0.6 + sparsity * 0.4        # score which serves as proof to determine which K should we choose

def fit_nmf_with_best_k(X):
    best = None
    for k in K_RANGE:
        nmf = NMF(n_components=k, init="nndsvda", random_state=42, max_iter=400)
        W = nmf.fit_transform(X); H = nmf.components_          # X = W*H
        s = score_topics(H)
        if (best is None) or (s > best["score"]): best = {"k": k, "model": nmf, "W": W, "H": H, "score": s}
    return best

# Main pipeline
def topic_pipeline(df: pd.DataFrame):
    df = df.copy()
    for col in ["content","review_id","thumbs_up_count","app_version","review_time"]:
        if col not in df.columns: raise KeyError(f"missing column: {col}")

    df["en_prop"] = df["content"].apply(english_prop)
    df = df[df["en_prop"] >= EN_PROP_MIN]
    df["text_clean"] = df["content"].apply(clean_text)
    df = df[df["text_clean"].str.len() >= 10]
    df = df.reset_index(drop=True)

    if df.empty: return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # Vectorize only English words (≥2 letters)
    vec = TfidfVectorizer(
        ngram_range=(1,3),
        min_df=MIN_DF, max_df=MAX_DF, max_features=MAX_FEATURES,
        stop_words=STOP_WORDS, sublinear_tf=True,
        token_pattern=r'(?u)\b[a-zA-Z]{2,}\b'
    )
    X = vec.fit_transform(df["text_clean"])
    terms = vec.get_feature_names_out()
    if X.shape[0] < min(K_RANGE): raise ValueError(f"Too few documents ({X.shape[0]}) for topic modeling.")

    best = fit_nmf_with_best_k(X)
    nmf, W, H, k = best["model"], best["W"], best["H"], best["k"]
    print(f"[Info] Selected K={k} (score={best['score']:.3f})")

    # Topic keywords (English phrases only)
    topics_rows, name_map = [], {}
    for t_idx in range(k):
        idx = np.argsort(H[t_idx])[::-1][:TOPN_WORDS]
        cand = [terms[i] for i in idx]
        kws_eng = [w for w in cand if _is_english_phrase(w) and w.lower() not in STOP_WORDS][:TOPN_WORDS]
        topics_rows.append({"topic": int(t_idx), "keywords": ", ".join(kws_eng)})
        name_map[t_idx] = auto_name(kws_eng)
    topics_df = pd.DataFrame(topics_rows)

    # Representative Reviews
    df["topic"] = W.argmax(axis=1)
    rows = []
    for t_idx in range(k):
        sub = df[df["topic"] == t_idx].copy()
        if sub.empty: continue
        pos = df.index.get_indexer(sub.index)
        weights = W[pos, t_idx]
        rep = weights * np.log1p(sub["thumbs_up_count"].fillna(0))
        sub = sub.assign(rep=rep)

        # Remove Duplicates
        uniq, seen = [], []
        for _, r in sub.sort_values("rep", ascending=False).iterrows():
            toks = _tokens_for_jaccard(r["content"])
            if any(jaccard(toks, s) >= DEDUP_JACCARD for s in seen): continue
            seen.append(toks); uniq.append(r)
        if not uniq: continue

        texts = [str(r["content"]) for r in uniq]
        rels  = [float(r["rep"]) for r in uniq]
        picks = mmr_select(texts, rels, EXAMPLES_PER_TP, lam=MMR_LAMBDA)
        for j in picks:
            r = uniq[j]
            rows.append({
                "topic": int(t_idx),
                "topic_name": name_map[t_idx],
                "review_id": r["review_id"],
                "thumbs_up_count": int(r.get("thumbs_up_count") or 0),
                "app_version": r.get("app_version"),
                "review_time": r.get("review_time"),
                "example_review": r["content"],
                "rep_score": float(r["rep"])
            })
    examples_df = pd.DataFrame(rows)

    # PM Summary
    total = len(df)
    summary_rows = []
    for t_idx in range(k):
        sub = df[df["topic"] == t_idx]
        if len(sub) == 0: continue
        share = len(sub) / total
        thumbs = sub["thumbs_up_count"].fillna(0)
        summary_rows.append({
            "topic": int(t_idx),
            "topic_name": name_map[t_idx],
            "share_pct": round(share * 100, 2),         # percentage of a given topic
            "thumbs_avg": float(thumbs.mean()),         # thumb-up average
            "thumbs_median": float(thumbs.median()),    # thumb-up median
            "sample_size": int(len(sub))
        })
    summary_df = pd.DataFrame(summary_rows).sort_values("topic")  

    return topics_df, examples_df, summary_df

# Table: create table + auto add missing columns
def ensure_tables(conn):
    with conn.cursor() as cs:
        cs.execute(f"""
        CREATE TABLE IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}.{TABLE_TOPICS} (
          run_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
          topic NUMBER,
          keywords STRING
        )""")
        cs.execute(f"""
        CREATE TABLE IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}.{TABLE_EXAMPLES} (
          run_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
          topic NUMBER,
          review_id STRING,
          thumbs_up_count NUMBER,
          app_version STRING,
          review_time TIMESTAMP_NTZ,
          example_review STRING
        )""")
        cs.execute(f"""
        CREATE TABLE IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY} (
          run_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
          topic NUMBER
        )""")
        # auto add missing columns
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_EXAMPLES} ADD COLUMN IF NOT EXISTS TOPIC_NAME  STRING")
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_EXAMPLES} ADD COLUMN IF NOT EXISTS REP_SCORE   FLOAT")
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY}  ADD COLUMN IF NOT EXISTS TOPIC_NAME   STRING")
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY}  ADD COLUMN IF NOT EXISTS SHARE_PCT    FLOAT")
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY}  ADD COLUMN IF NOT EXISTS THUMBS_AVG   FLOAT")
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY}  ADD COLUMN IF NOT EXISTS THUMBS_MEDIAN FLOAT")
        cs.execute(f"ALTER TABLE {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY}  ADD COLUMN IF NOT EXISTS SAMPLE_SIZE  NUMBER")

def to_iso_ntz(x):
    if pd.isna(x): return None
    ts = pd.to_datetime(x, errors="coerce")
    if pd.isna(ts): return None
    try: ts = ts.tz_convert(None)
    except Exception: pass
    return ts.strftime("%Y-%m-%d %H:%M:%S")

def write_back(conn, topics_df, examples_df, summary_df):
    if not examples_df.empty and "review_time" in examples_df.columns:
        examples_df = examples_df.copy()
        examples_df["review_time"] = examples_df["review_time"].apply(to_iso_ntz)

    with conn.cursor() as cs:
        for tbl in [TABLE_TOPICS, TABLE_EXAMPLES, TABLE_SUMMARY]:
            cs.execute(f"DELETE FROM {DB_NAME}.{SCHEMA_NAME}.{tbl} WHERE DATE(run_at)=CURRENT_DATE()")

        if not topics_df.empty:
            cs.executemany(
                f"INSERT INTO {DB_NAME}.{SCHEMA_NAME}.{TABLE_TOPICS}(topic, keywords) VALUES (%(topic)s, %(keywords)s)",
                topics_df.to_dict("records"))

        if not examples_df.empty:
            cs.executemany(
                f"""INSERT INTO {DB_NAME}.{SCHEMA_NAME}.{TABLE_EXAMPLES}
                (topic, topic_name, review_id, thumbs_up_count, app_version, review_time, example_review, rep_score)
                VALUES (%(topic)s, %(topic_name)s, %(review_id)s, %(thumbs_up_count)s, %(app_version)s, %(review_time)s, %(example_review)s, %(rep_score)s)""",
                examples_df.to_dict("records"))

        if not summary_df.empty:
            cs.executemany(
                f"""INSERT INTO {DB_NAME}.{SCHEMA_NAME}.{TABLE_SUMMARY}
                (topic, topic_name, share_pct, thumbs_avg, thumbs_median, sample_size)
                VALUES (%(topic)s, %(topic_name)s, %(share_pct)s, %(thumbs_avg)s, %(thumbs_median)s, %(sample_size)s)""",
                summary_df.to_dict("records"))

# Main Process
def main():
    cfg  = load_env()
    conn = connect_sf(cfg)

    ctx = pd.read_sql("SELECT CURRENT_USER() u, CURRENT_ROLE() r, CURRENT_DATABASE() db, CURRENT_SCHEMA() sch", conn)
    print(ctx.to_string(index=False))

    df = fetch_df(conn)
    print(f"Loaded {len(df):,} low-score rows (before filters)")

    topics_df, examples_df, summary_df = topic_pipeline(df)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    p1, p2, p3 = (f"topics_keywords_{ts}.csv",
                  f"topics_examples_{ts}.csv",
                  f"topics_summary_{ts}.csv")
    topics_df.to_csv(p1, index=False)
    examples_df.to_csv(p2, index=False)
    summary_df.to_csv(p3, index=False)
    print(f"Saved: {p1}, {p2}, {p3}")

    ensure_tables(conn)
    write_back(conn, topics_df, examples_df, summary_df)
    print(f"Wrote results to {DB_NAME}.{SCHEMA_NAME}.{TABLE_TOPICS} / {TABLE_EXAMPLES} / {TABLE_SUMMARY}")

    conn.close(); print("Done.")

if __name__ == "__main__":
    main()


/var/folders/0n/f6xlkj4n0v18gt_r7s7t1j700000gn/T/ipykernel_38355/1413911622.py:343: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ctx = pd.read_sql("SELECT CURRENT_USER() u, CURRENT_ROLE() r, CURRENT_DATABASE() db, CURRENT_SCHEMA() sch", conn)
/var/folders/0n/f6xlkj4n0v18gt_r7s7t1j700000gn/T/ipykernel_38355/1413911622.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


          U            R                 DB     SCH
MATTCAO2001 ACCOUNTADMIN CHATGPT_REVIEWS_DB STAGING
Loaded 5,883 low-score rows (before filters)
[Info] Selected K=12 (score=0.925)
Saved: topics_keywords_20250819_185531.csv, topics_examples_20250819_185531.csv, topics_summary_20250819_185531.csv
Wrote results to CHATGPT_REVIEWS_DB.STAGING.TOPICS_KEYWORDS / TOPICS_EXAMPLES / TOPICS_SUMMARY
Done.
